In [1]:
import os
from contextlib import closing
import threading
import requests
import pandas as pd
from pdf2image import convert_from_path

In [2]:
headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
}

In [3]:
img_dir = "image/"
thread_num = 50
timeout = 30
len_to_download = 0

if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [4]:
def download(img_url,img_name):
    img_name = img_name +".pdf"
    if os.path.isfile(os.path.join(img_dir,img_name)):
        return
    with closing(requests.get(img_url,stream=True,headers=headers,timeout=timeout)) as r:
        rc = r.status_code
        if 299 < rc or rc < 200:
            print ('returnCode%s\t%s' % (rc, img_url))
            return
        content_length = int(r.headers.get('content-length', '0'))
        if content_length == 0:
            print ('size0\t%s' % img_url)
            return
        try:
            with open(os.path.join(img_dir,img_name),"wb") as f:
                for data in r.iter_content(8192):
                    f.write(data)
#             jpg_name = img_name + ".jpg"    
#             page = convert_from_path(os.path.join(img_dir,img_name))
#             page[0].save(os.path.join(img_dir,jpg_name),"PNG")
#                 image = convert_from_path(os.path.join(img_dir,i))
#                 image[0].save(os.path.join(img_dir,name),"PNG")
        except:
            print('savefail\t%s' % img_url)

In [5]:
lock = threading.Lock()
fail_urls = []
def loop(imgs):
#     print ('thread %s is running...' % threading.current_thread().name)
    while True:
        try:
            with lock:
                img_url,img_name = next(imgs)
#                 print(img_name)
        except StopIteration:
            break
        try:
            download(img_url,img_name)
        except:
            global fail_urls
            fail_urls.append((img_url,img_name))
            print("exceptfail\t%s"%img_url)
    print("thread %s is end ..."% threading.current_thread().name)

In [6]:
def get_img_url_generate():
    imgs = []
    df = pd.read_excel("demo.xlsx")
    global len_to_download
    len_to_download = len(df)
    df = df[["pdf下载链接","专利号"]]
    for i,j in zip(df.pdf下载链接,df.专利号):
        imgs = []
        img_url = i
        img_name = j
        imgs.append(img_url)
        imgs.append(img_name)
        try:
            if img_url:
                yield imgs
        except:
            break

In [7]:
imgs = get_img_url_generate()

In [8]:
for i in range(0,thread_num):
    t = threading.Thread(target=loop,name="LoopThread %s" %i,args=(imgs,))
    t.start()
len_have_download = len(os.listdir(img_dir))

全部下载成功


In [18]:
fail_urls

[('https://pdfpiw.uspto.gov/58/176/D08/1.pdf', 'D0817658'),
 ('https://pdfpiw.uspto.gov/70/466/065/1.pdf', '06546670')]

In [19]:
def tryagain_fail(fail_urls):
    for img_url, img_name in fail_urls:
        img_name = img_name +".pdf"
        if os.path.isfile(os.path.join(img_dir,img_name)):
            return
        r = requests.get(img_url,stream=True,headers=headers,timeout=timeout)
        rc = r.status_code
        if 299 < rc or rc < 200:
            print ('returnCode%s\t%s' % (rc, img_url))
            return
        content_length = int(r.headers.get('content-length', '0'))
        if content_length == 0:
            print ('size0\t%s' % img_url)
            return
        try:
            with open(os.path.join(img_dir,img_name),"wb") as f:
                for data in r.iter_content(8192):
                    f.write(data)
        except:
            print('savefail\t%s' % img_url)

In [20]:
tryagain_fail(fail_urls)

In [21]:
len(os.listdir(img_dir))

256

In [22]:
for i in os.listdir(img_dir):
    name = i.split(".")[0] + ".png"
    try:
        image = convert_from_path(os.path.join(img_dir,i))
        image[0].save(os.path.join(img_dir,name),"PNG")
        print(os.path.join(img_dir,i))
    except:
        print("出错 --",name)

image/03930334.pdf
image/04026220.pdf
image/04040208.pdf
image/04098483.pdf
image/04149339.pdf
image/04159094.pdf
image/04170843.pdf
image/04187996.pdf
image/04262873.pdf
image/04314646.pdf
image/04337916.pdf
image/04349864.pdf
image/04385742.pdf
image/04446653.pdf
image/04449324.pdf
image/04463855.pdf
image/04574521.pdf
image/04658540.pdf
image/04666115.pdf
image/04674415.pdf
image/04770303.pdf
image/04771573.pdf
image/04779377.pdf
image/04779378.pdf
image/04811475.pdf
image/04825589.pdf
image/04845602.pdf
image/04882875.pdf
image/04908982.pdf
image/04912876.pdf
image/05020274.pdf
image/05037049.pdf
image/05050339.pdf
image/05052149.pdf
image/05088234.pdf
image/05251399.pdf
image/05309670.pdf
image/05315784.pdf
image/05345712.pdf
image/05347751.pdf
image/05360193.pdf
image/05363591.pdf
image/05450692.pdf
image/05502922.pdf
image/05546698.pdf
image/05606824.pdf
image/05673511.pdf
image/05743045.pdf
image/05806241.pdf
image/05964057.pdf
image/06000172.pdf
image/06029396.pdf
image/060328

In [14]:
len_to_download

256